# BentoML Example:  Text Classification with Keras

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&t=event&ec=nb&ea=open&dp=keras-text-classification&dt=keras-text-classification)

Example based on Tensorflow's text-classification tutorial. [Here](https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/basic_text_classification.ipynb) is the original tutorial notebook.

In [ ]:
!pip install -I bentoml
!pip install tensorflow==1.13.1
!pip install numpy==1.16.1

In [ ]:
from __future__ import absolute_import, division, print_function

import numpy as np
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

import bentoml

## Download the IMDB dataset

In [ ]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

In [ ]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(encoded_text):
    return ' '.join([reverse_word_index.get(i, '?') for i in encoded_text])

In [ ]:
train_data = keras.preprocessing.sequence.pad_sequences(train_data,
                                                        value=word_index["<PAD>"],
                                                        padding='post',
                                                        maxlen=256)

test_data = keras.preprocessing.sequence.pad_sequences(test_data,
                                                       value=word_index["<PAD>"],
                                                       padding='post',
                                                       maxlen=256)

## Build the model

In [ ]:
# input shape is the vocabulary count used for the movie reviews (10,000 words)
vocab_size = 10000

model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

## Create a validation set

In [ ]:
x_val = train_data[:10000]
partial_x_train = train_data[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

## Train the model

In [ ]:
model.fit(partial_x_train,
                    partial_y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)

## Evaluate the model

In [ ]:
results = model.evaluate(test_data, test_labels)

print(results)

In [ ]:
model.summary()

## Define BentoML service

In [ ]:
%%writefile text_classification_service.py
import pandas as pd
import numpy as np
from tensorflow import keras
from bentoml import api, env, BentoService, artifacts
from bentoml.artifact import TfKerasModelArtifact, PickleArtifact
from bentoml.handlers import JsonHandler

@artifacts([
    TfKerasModelArtifact('model'),
    PickleArtifact('word_index')
])
@env(conda_dependencies=['tensorflow', 'numpy', 'pandas'])
class TextClassificationService(BentoService):
   
    def word_to_index(self, word):
        if word in self.artifacts.word_index:
            return self.artifacts.word_index[word]
        else:
            return self.artifacts.word_index["<UNK>"]
    
    @api(JsonHandler)
    def predict(self, parsed_json):
        """
        """
        text = parsed_json['text']
        
        sequence = keras.preprocessing.text.hashing_trick(
            text,
            256,
            hash_function=self.word_to_index,
            filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
            lower=True,
            split=' ')

        return self.artifacts.model.predict(np.expand_dims(sequence, 0))

## Save new BentoML service archive

In [ ]:
from text_classification_service import TextClassificationService

svc = TextClassificationService.pack(model=model, word_index=word_index)
saved_path = svc.save('/tmp/bento')
print(saved_path)

## Test packed BentoML service

In [ ]:
svc.predict({ 'text': 'the best movie I have ever seen' })

In [ ]:
svc.predict({ 'text': 'This is a bad movie' })

# Use Case 1 - load BentoML service archive in python

BentoML user can use `bentoml#load` method to load in an exported model service directly:

In [ ]:
import bentoml

bento_svc = bentoml.load(saved_path)

In [ ]:
bento_svc.predict({ 'text': 'the best movie I have ever seen' })

In [ ]:
bento_svc.predict({ 'text': 'This is a bad movie' })

# Use Case 2 - Run BentoML service as REST API server
A saved BentoML service archive can be loaded as a REST API server with bentoml package

##### * For demo purpurse, copy generated model to ./model folder

In [ ]:
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

In [ ]:
!bentoml serve ./model

*Copy following command to make a curl request to Rest API server*
```bash
curl -i \
--header "Content-Type: application/json" \
--request POST \
--data '{"text": "some new text, sweet noodles"}' \
localhost:5000/predict
```

# Use Case 3 - python package

BentoML user can pip install exported BentoML Archive ans use it as a regular python package.
Simply do `pip install $SAVED_PATH`:

In [ ]:
import subprocess
print(subprocess.check_output(['pip', 'install', saved_path]).decode())

In [ ]:
import TextClassificationService

installed_svc = TextClassificationService.load()

In [ ]:
installed_svc.predict({ 'text': 'the best movie I have ever seen' })

In [ ]:
installed_svc.predict({ 'text': 'This is a bad movie' })

# Use Case 3 - CLI access

After `pip install` above, bentoml also provides a CLI access to the install module:

In [ ]:
!TextClassificationService --help

### Print model service information:

In [ ]:
!TextClassificationService info

### Run 'predict' api with json data:

In [ ]:
!TextClassificationService predict --input='{"text": "bad movie"}'